In [ ]:
import rasterio
import numpy as np
import os
import cv2
from tqdm import tqdm
import json
import os
import numpy as np
from PIL import Image

In [ ]:
# Charleston flood
before_path = # directtion to "Before_SAR_Image" exported from GEE
after_path = # directtion to "After_SAR_Image" exported from GEE
mask_path = # directtion to "Flood_Mask" exported from GEE
permanent_water_path = # directtion to "Permanent_Water_Layer" exported from GEE

with rasterio.open(before_path) as before_src:
    before_img = before_src.read(1)

with rasterio.open(after_path) as after_src:
    after_img = after_src.read(1)

with rasterio.open(mask_path) as mask_src:
    mask_img = mask_src.read(1)

with rasterio.open(permanent_water_path) as permanent_water_src:
    permanent_water_img = permanent_water_src.read(1)




In [ ]:
def create_chips(image, chip_size, step):
    h, w = image.shape
    chips = []
    for i in range(0, h - chip_size + 1, step):
        for j in range(0, w - chip_size + 1, step):
            chip = image[i:i + chip_size, j:j + chip_size]
            chips.append(chip)
    return chips

chip_size = 256
step = chip_size // 2  # 50% overlap

before_chips = create_chips(before_img, chip_size, step)
after_chips = create_chips(after_img, chip_size, step)
mask_chips = create_chips(mask_img, chip_size, step)
water_chips = create_chips(permanent_water_img, chip_size, step)

In [ ]:
def save_chips(chips, output_dir, image_path):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    prefix = os.path.basename(image_path).split('_')[0]
    for idx, chip in enumerate(tqdm(chips)):
        # Normalize chip to 0-255
        chip = np.clip(chip, a_min=0, a_max=None)  # Remove negative values (if any)
        chip_normalized = (chip - chip.min()) / (chip.max() - chip.min()) * 255  # Normalize to [0, 255]
        chip_normalized = chip_normalized.astype(np.uint8)

        # Save the chip as PNG
        output_path = os.path.join(output_dir, f"{prefix}_{idx}.png")
        cv2.imwrite(output_path, chip_normalized)

save_chips(before_chips, 'chips/before', before_path)
save_chips(after_chips, 'chips/after', after_path)
save_chips(mask_chips, 'chips/mask', mask_path)
save_chips(water_chips, 'chips/water', permanent_water_path)


In [ ]:
import os
import numpy as np
from PIL import Image

def filter_flooded_chips(mask_dir, image_dir_before, image_dir_after, output_dir, flood_threshold=0.1):
    os.makedirs(output_dir, exist_ok=True)
    valid_chips = []

    for mask_file in os.listdir(mask_dir):
        mask_path = os.path.join(mask_dir, mask_file)
        mask = np.array(Image.open(mask_path))

        flood_percentage = np.sum(mask > 0) / mask.size

        if flood_percentage > flood_threshold:  # Keep chips with significant flooding
            valid_chips.append(mask_file)

            for sub_dir, chip_name in zip(
                [image_dir_before, image_dir_after, mask_dir],
                ["before", "after", "mask"]
            ):
                chip_path = os.path.join(sub_dir, mask_file)
                output_path = os.path.join(output_dir, chip_name)
                os.makedirs(output_path, exist_ok=True)
                Image.open(chip_path).save(os.path.join(output_path, mask_file))

    print(f"Kept {len(valid_chips)} useful chips out of {len(os.listdir(mask_dir))} total.")

    # Save valid file names to a text file
    with open(os.path.join(output_dir, "valid_files.txt"), "w") as f:
        for chip in valid_chips:
            f.write(f"{chip}\n")

    return valid_chips

# Example usage
filter_flooded_chips(
    mask_dir="chips/mask",
    image_dir_before="chips/before",
    image_dir_after="chips/after",
    output_dir="chips/floodedChips10P",
    flood_threshold=0.1  # 10% flood threshold
)


In [ ]:
import os
import json

# Step 8: Creating JSON metadata file
def create_metadata(chips, output_dir, image_path, corresponding_images):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    metadata = []
    prefix = os.path.basename(image_path).split('_')[0]

    for idx, chip in enumerate(chips):
        entry = {
            "id": f"{prefix}_{idx}",
            "image_path": os.path.join(output_dir, f"{prefix}_{idx}.png"),
            "source_image": image_path,
            "before_image": corresponding_images.get("before"),
            "after_image": corresponding_images.get("after"),
            "coordinates": (idx // 256, idx % 256)  # Example coordinate, adjust as needed
        }
        metadata.append(entry)

    # Save the metadata to a JSON file
    json_file_path = os.path.join(output_dir, f"{prefix}_metadata.json")
    with open(json_file_path, 'w') as json_file:
        json.dump(metadata, json_file, indent=4)


mask_path = 'chips/floodedChips10P/mask'
before_path = 'chips/floodedChips10P/before'
after_path = 'chips/floodedChips10P/after'
# Use this modified function to create metadata for all categories, while providing corresponding images
create_metadata(mask_chips, 'chips/floodedChips10P/mask', mask_path, {"before": before_path, "after": after_path})
#create_metadata(water_chips, 'chips/floodedChips10P/water', permanent_water_path, {"before": before_path, "after": after_path})
create_metadata(before_chips, 'chips/floodedChips10P/before', before_path, {"mask": mask_path, "water": permanent_water_path})
create_metadata(after_chips, 'chips/floodedChips10P/after', after_path, {"mask": mask_path, "water": permanent_water_path})
